In [36]:
## Load the code

include("ratio_of_function_values.jl");

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-06


In [37]:
## Parameters

μ = 0.5
L = 1
R = 1 # min(R_term_1,R_term_2)
bigM = 1e6
η_2 = 1 # should be 1 for PRP, 0 for FR
c_0_input_initial_given = (L+μ)^2/(4*μ*L) 
# this means ||d_{k-1}||^2 <= c_0*||g_{k-1}||^2, should be something > 1, 
# valid selection for PRP is (1+(L/μ)^2) based on Polyak's result
# for FR, it can be any number greater than 1

# for finding function value ratio
c_f_input, c_x_input, c_g_input, c_p_input =  1, 0, 0, 0
ι_f_input, ι_x_input, ι_g_input, ι_p_input = 1, 0, 0, 0

(1, 0, 0, 0)

In [38]:
## Warm-start point

G_ws, Ft_ws, L_cholesky_ws, γ_kM1_ws, β_kM1_ws = feasible_sol_generator_Lyapunov_Ada_PEP(μ, L, R, η_2; c_f = c_f_input, c_x = c_x_input, c_g = c_g_input, c_p = c_p_input, ι_f = ι_f_input, ι_x = ι_x_input, ι_g = ι_g_input, ι_p = ι_p_input)

## Create the index set of points

k = 1; ⋆ = -1;
I_N_star = [⋆; k-1; k]
idx_set_λ_ws_effective = index_set_constructor_for_dual_vars_full(I_N_star);

[ Info: conjugacy condition is satisfied


In [39]:
## Generate locally optimal solution

G_loc_opt, Ft_loc_opt, L_cholesky_loc_opt, γ_kM1_loc_opt, β_kM1_loc_opt, t_loc_opt, lower_bound_var_loc_opt, upper_bound_var_loc_opt, solve_time_loc_opt = Ratio_of_Function_Values_Model(
	# different parameters to be used
    # -------------------------------
    μ, L, R,
    # warm-start points
    # -----------------
    idx_set_λ_ws_effective, G_ws, Ft_ws, L_cholesky_ws, γ_kM1_ws,  β_kM1_ws;
	# different options
	# -----------------
	# [🐘 ] solver type
	solver = LocalOptSolver,
	# the solvers are:
	# GlobalOptSolver to find the globally optimal solution (uses gurobi)
	# LocalOptSolver to find the locally optimal solution (uses KNITRO)
	# BoundCompSolver to find valid bounds on the decision variables (uses Alpine)
	# [🐯 ] cost coefficients
    c_f = c_f_input, c_x = c_x_input, c_g = c_g_input, c_p = c_p_input,
	ι_f = ι_f_input, ι_x = ι_x_input, ι_g = ι_g_input, ι_p = ι_p_input,
	show_output = :on, # options are :on and :off
	lower_bound_opt_val = 0,
	upper_bound_opt_val = Inf,
	impose_pattern_interpolation = :off,
	impose_pattern_noninterpolation = :off,
	bound_impose = :off, # # options are :generic, and :off
	bound_M = Inf,
	lower_bound_var = [], # lower bound vector on the decision variables
	upper_bound_var = [], # upper bound vector on the decision variables
	PSDness_modeling = :exact, # options are :exact and :through_ϵ and :lazy_constraint_callback
    ϵ_tol_feas = 1e-6, # feasiblity tolerance for minimum eigenvalue of G
    maxCutCount=1e6, # number of lazy cuts when we use lazy constraints for modeling G = L_cholesky*L_cholesky
	PRP_Plus = :off, # options are :positive_case, :negative_case and :off
    c_0_input_initial = c_0_input_initial_given,
	η_2_input = η_2
	)

## Set the warm-starting points for the later stage

G_ws, Ft_ws, L_cholesky_ws, γ_kM1_ws, β_kM1_ws, t_ws, lower_bound_var_ws, upper_bound_var_ws, solve_time_ws = G_loc_opt, Ft_loc_opt, L_cholesky_loc_opt, γ_kM1_loc_opt, β_kM1_loc_opt, t_loc_opt, lower_bound_var_loc_opt, upper_bound_var_loc_opt, solve_time_loc_opt

## Compute bound

bound_M_G_loc_opt = 3*maximum(G_loc_opt[i,i] for i in 1:size(G_loc_opt,1));

bound_M_F_loc_opt = 3*maximum(Ft_loc_opt[i] for i in 1:size(Ft_loc_opt,1));

bound_M_loc_opt = maximum([bound_M_G_loc_opt, bound_M_F_loc_opt]);

[ Info: [🎉 ] defining the variables
[ Info: no bound imposed
[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎢 ] adding the constraints
[ Info: coninterpol ℓ = i_j_idx(-1, 0)
[ Info: coninterpol ℓ = i_j_idx(-1, 1)
[ Info: coninterpol ℓ = i_j_idx(0, -1)
[ Info: coninterpol ℓ = i_j_idx(0, 1)
[ Info: coninterpol ℓ = i_j_idx(1, -1)
[ Info: coninterpol ℓ = i_j_idx(1, 0)



           Academic License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 13.1.0

Knitro presolve eliminated 82 variables and 146 constraints.

algorithm:               1
convex:                  0
datacheck:               0
hessian_no_f:            1
honorbnds:               1
strat_warm_start:        1
The problem is identified as a QCQP.
Knitro shifted start point further inside presolved bounds (5 variables).
Knitro fixing 78 variables eliminated from the presolve.

Problem Characteristics                                 (   Presolved)
-----------------------
Objective goal:  Maximize
Objective type:  linear
Number of variables:                                111 (          29)
    bounded below only:                               2 (           8)
    bounded above only:                               0 (           0)
    bounded below and above:                          0 (           2)
    fixed:                                            6 (           0)
    free:     

[ Info: [🎇 ] adding objective
[ Info: [👲 ] warm-start values for all the variables
[ Info: [🙌 	🙏 ] model building done, starting the optimization process
[ Info: [😻 ] optimal solution found done, store the solution
[ Info: [💹 ] warm-start objective value = 0.004890027894322182, and objective value of found solution = 0.3600002727156568
[ Info: IMPORTANT INFO
[ Info: ------------------------------------------
[ Info: ||g_{k-1}||^2 =  1.1999998142876516
[ Info: ||p_{k-1}||^2 = 1.349999791073655
[ Info: NCG-PEP Theory PRP ( f_{k+1} - f_⋆ ) / ( f_k - f_⋆ ) = 0.36
[ Info: NCG-PEP Numerical ( f_{k+1} - f_⋆ ) / ( f_k - f_⋆ )= 0.3600002727156568
[ Info: ------------------------------------------
[ Info: [💀 ] The dual multipliers for interpolation inequalities are: [-0.24004714599534238, -0.4002675048847742, -0.00012546309130551334, -0.6000282821171736, -0.00018912415651942192, -0.0001066658954143929]


In [40]:
## Compute globally optimal solution

G_glb_opt, Ft_glb_opt, L_cholesky_glb_opt, γ_kM1_glb_opt, β_kM1_glb_opt, t_glb_opt, lower_bound_var_glb_opt, upper_bound_var_glb_opt, solve_time_glb_opt = Ratio_of_Function_Values_Model(
	# different parameters to be used
    # -------------------------------
    μ, L, R,
    # warm-start points
    # -----------------
    idx_set_λ_ws_effective, G_ws, Ft_ws, L_cholesky_ws, γ_kM1_ws,  β_kM1_ws;
	# different options
	# -----------------
	# [🐘 ] solver type
	solver = GlobalOptSolver,
	# the solvers are:
	# GlobalOptSolver to find the globally optimal solution (uses gurobi)
	# LocalOptSolver to find the locally optimal solution (uses KNITRO)
	# BoundCompSolver to find valid bounds on the decision variables (uses Alpine)
	# [🐯 ] cost coefficients
    c_f = c_f_input, c_x = c_x_input, c_g = c_g_input, c_p = c_p_input,
	ι_f = ι_f_input, ι_x = ι_x_input, ι_g = ι_g_input, ι_p = ι_p_input,
	show_output = :on, # options are :on and :off
	lower_bound_opt_val = 0,
	upper_bound_opt_val = Inf,
	impose_pattern_interpolation = :off,
	impose_pattern_noninterpolation = :off,
	bound_impose = :generic, # # options are :generic, :alpine, and :off
	bound_M = bound_M_loc_opt,
	lower_bound_var = [], # lower bound vector on the decision variables
	upper_bound_var = [], # upper bound vector on the decision variables
	PSDness_modeling = :lazy_constraint_callback, # options are :exact and :through_ϵ and :lazy_constraint_callback
    ϵ_tol_feas = 1e-6, # feasiblity tolerance for minimum eigenvalue of G
    maxCutCount=1e6, # number of lazy cuts when we use lazy constraints for modeling G = L_cholesky*L_cholesky
	PRP_Plus = :off, # options are :positive_case, :negative_case and :off
	c_0_input_initial = c_0_input_initial_given,
	η_2_input = η_2
	)

## Set the warm-starting points for the later stage

G_ws, Ft_ws, L_cholesky_ws, γ_kM1_ws, β_kM1_ws, t_ws, lower_bound_var_ws, upper_bound_var_ws, solve_time_ws = G_glb_opt, Ft_glb_opt, L_cholesky_glb_opt, γ_kM1_glb_opt, β_kM1_glb_opt, t_glb_opt, lower_bound_var_glb_opt, upper_bound_var_glb_opt, solve_time_glb_opt

Set parameter NonConvex to value 2
Set parameter MIPFocus to value 3
Set parameter MIPGap to value 0.01
Set parameter Presolve to value 2
Set parameter MIPFocus to value 1


[ Info: [🎉 ] defining the variables
[ Info: [💣 ] imposing bound on the the entries of G, F based on Lyapunov function choice
[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎢 ] adding the constraints
[ Info: coninterpol ℓ = i_j_idx(-1, 0)
[ Info: coninterpol ℓ = i_j_idx(-1, 1)
[ Info: coninterpol ℓ = i_j_idx(0, -1)
[ Info: coninterpol ℓ = i_j_idx(0, 1)
[ Info: coninterpol ℓ = i_j_idx(1, -1)
[ Info: coninterpol ℓ = i_j_idx(1, 0)
[ Info: [🎇 ] adding objective
[ Info: [👲 ] warm-start values for all the variables
[ Info: [🙌 	🙏 ] model building done, starting the optimization process
[ Info: [😻 ] optimal solution found done, store the solution


Set parameter NonConvex to value 2
Set parameter MIPFocus to value 1
Set parameter MIPGap to value 0.01
Set parameter Presolve to value 2
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 209 rows, 111 columns and 1088 nonzeros
Model fingerprint: 0xd3d10540
Model has 66 quadratic constraints
Coefficient statistics:
  Matrix range     [8e-07, 4e+00]
  QMatrix range    [5e-01, 2e+00]
  QLMatrix range   [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 1e+01]
  RHS range        [1e+00, 1e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 108 rows and 78 columns

Continuous model is non-convex -- solving as a MIP


Loaded user MIP start with objective 0.36

Presolve removed 115 rows and 87 columns
Presolve time: 0.00s
Presolved: 113 rows, 27 columns, 663 nonzeros
Presolved mode

[ Info: [💹 ] warm-start objective value = 0.3600002733457846, and objective value of found solution = 0.3600002733457846
[ Info: IMPORTANT INFO
[ Info: ------------------------------------------
[ Info: ||g_{k-1}||^2 =  1.1999998142876516
[ Info: ||p_{k-1}||^2 = 1.349999791073608
[ Info: NCG-PEP Theory PRP ( f_{k+1} - f_⋆ ) / ( f_k - f_⋆ ) = 0.36
[ Info: NCG-PEP Numerical ( f_{k+1} - f_⋆ ) / ( f_k - f_⋆ )= 0.3600002733457846
[ Info: ------------------------------------------


([3.5988028435692203 1.9997006195072409 0.7197636879990472 1.7997752406277492; 1.9997006195072409 1.1999998142876516 0.23998103585594127 1.1999998142876516; 0.7197636879990472 0.23998103585594127 0.431931601491017 0.0; 1.7997752406277492 1.1999998142876516 0.0 1.349999791073608], [1.0, 0.3600002733457846], [1.8970511033831643 0.0 0.0 0.0; 1.0541100163371706 0.2980804526239485 0.0 0.0; 0.37941183503415654 -0.5366361102097084 0.0 0.0; 0.9487225839277059 0.6707646870663218 0.0 0.0], 1.0667930000764576, 0.1599588296094507, 0.3600002733457846, Any[], Any[], 0.30562520027160645)

In [41]:
## Polish the globally optimal solution

G_glb_opt_plshd, Ft_glb_opt_plshd, L_cholesky_glb_opt_plshd, γ_kM1_glb_opt_plshd, β_kM1_glb_opt_plshd, t_glb_opt_plshd, lower_bound_var_glb_opt_plshd, upper_bound_var_glb_opt_plshd, solve_time_glb_opt_plshd = Ratio_of_Function_Values_Model(
	# different parameters to be used
    # -------------------------------
    μ, L, R,
    # warm-start points
    # -----------------
    idx_set_λ_ws_effective, G_ws, Ft_ws, L_cholesky_ws, γ_kM1_ws,  β_kM1_ws;
	# different options
	# -----------------
	# [🐘 ] solver type
	solver = LocalOptSolver,
	# the solvers are:
	# GlobalOptSolver to find the globally optimal solution (uses gurobi)
	# LocalOptSolver to find the locally optimal solution (uses KNITRO)
	# BoundCompSolver to find valid bounds on the decision variables (uses Alpine)
	# [🐯 ] cost coefficients
    c_f = c_f_input, c_x = c_x_input, c_g = c_g_input, c_p = c_p_input,
	ι_f = ι_f_input, ι_x = ι_x_input, ι_g = ι_g_input, ι_p = ι_p_input,
	show_output = :on, # options are :on and :off
	lower_bound_opt_val = 0,
	upper_bound_opt_val = Inf,
	impose_pattern_interpolation = :off,
	impose_pattern_noninterpolation = :off,
	bound_impose = :off, # # options are :generic, :alpine, and :off
	bound_M = bound_M_loc_opt,
	lower_bound_var = [], # lower bound vector on the decision variables
	upper_bound_var = [], # upper bound vector on the decision variables
	PSDness_modeling = :exact, # options are :exact and :through_ϵ and :lazy_constraint_callback
    ϵ_tol_feas = 1e-4, # feasiblity tolerance for minimum eigenvalue of G
    maxCutCount=1e6, # number of lazy cuts when we use lazy constraints for modeling G = L_cholesky*L_cholesky
	PRP_Plus = :off, # options are :positive_case, :negative_case and :off
	c_0_input_initial = c_0_input_initial_given,
	η_2_input = η_2
	)



           Academic License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 13.1.0

Knitro presolve eliminated 82 variables and 146 constraints.

algorithm:               1
convex:                  0
datacheck:               0
hessian_no_f:            1
honorbnds:               1
strat_warm_start:        1
The problem is identified as a QCQP.
Knitro shifted start point further inside presolved bounds (2 variables).
Knitro fixing 78 variables eliminated from the presolve.

Problem Characteristics                                 (   Presolved)
-----------------------
Objective goal:  Maximize
Objective type:  linear
Number of variables:                                111 (          29)
    bounded below only:                               2 (           8)
    bounded above only:                               0 (           0)
    bounded below and above:                          0 (           2)
    fixed:                                            6 (           0)
    free:     

[ Info: [🎉 ] defining the variables
[ Info: no bound imposed
[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎢 ] adding the constraints
[ Info: coninterpol ℓ = i_j_idx(-1, 0)
[ Info: coninterpol ℓ = i_j_idx(-1, 1)
[ Info: coninterpol ℓ = i_j_idx(0, -1)
[ Info: coninterpol ℓ = i_j_idx(0, 1)
[ Info: coninterpol ℓ = i_j_idx(1, -1)
[ Info: coninterpol ℓ = i_j_idx(1, 0)
[ Info: [🎇 ] adding objective
[ Info: [👲 ] warm-start values for all the variables
[ Info: [🙌 	🙏 ] model building done, starting the optimization process
[ Info: [😻 ] optimal solution found done, store the solution
[ Info: [💹 ] warm-start objective value = 0.3600002733457846, and objective value of found solution = 0.3600000071640856
[ Info: IMPORTANT INFO
[ Info: ------------------------------------------
[ Info: ||g_{k-1}||^2 =  1.1999999922394473
[ Info: ||p_{k-1}||^2 = 1.349999991269378
[ Info: NCG-PEP Theory PRP ( f_{k+1} - f_⋆ ) / ( f_k - f_⋆ ) = 0.36
[ Info: NCG-PEP Numerical ( f_{k+1} - f_⋆

([3.599667497372845 1.9999168704316874 0.7199346352648768 1.7999376495053419; 1.9999168704316874 1.1999999922394473 0.23999483520655507 1.1999999922394475; 0.7199346352648768 0.23999483520655507 0.4319814085789792 0.0; 1.7999376495053419 1.1999999922394475 0.0 1.349999991269378], [1.0, 0.36000000707305213], [1.8972789723623942 0.0 0.0 0.0; 1.0540974193972938 0.29812518305274716 0.0 0.0; 0.379456392067821 -0.5366509631993033 0.0 0.0; 0.9486942485457361 0.6708048998404537 0.0 0.0], 1.0667010860346273, 0.1599888121763658, 0.3600000071640856, Any[], Any[], 0.0013470000121742487)

In [42]:
## Solve primal SDP by fixing the worst-case stepsize found from NCG-PEP

p_star, G_star, Ft_star, L_cholesky_star = SDP_with_fixed_stepsize_ratio(
	# Inputs
			# ======
			μ, L, R, γ_kM1_glb_opt_plshd, β_kM1_glb_opt_plshd;
			# [🐯 ] cost coefficients
			# Options
			# =======
			η_2_input = η_2, # controls PRP or FR, if η = 1 => PRP, and if η = 0 => FR
			c_0_input_initial = c_0_input_initial_given, # decides ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
			show_output = :on, # options are :on and :off
			ϵ_tolerance = 1e-8,
			reduce_index_set_for_λ = :off
	)

Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 14              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 3               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Obj

[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎉 ] defining the variables
[ Info: [🎢 ] adding the constraints
[ Info: [👲 ] warm-start values for all the variables
[ Info: Time to optimize
[ Info: [😈 ] Optimal solution found for solve_primal_with_known_stepsizes
[ Info: ( f_{k+1} - f_⋆ ) / ( f_k - f_⋆ )= 0.3599999909960635
[ Info: NCG-PEP Theory PRP ( f_{k+1} - f_⋆ ) / ( f_k - f_⋆ ) = 0.36


(0.3599999909960635, [3.5993309917424408 1.9998327659975963 0.7198841629965799 1.7998745809747128; 1.9998327659975963 1.2000000278566978 0.23999484921126676 1.2000000227503251; 0.7198841629965799 0.23999484921126676 0.4319814325670142 3.5465617642785853e-12; 1.7998745809747128 1.2000000227503251 3.5465617642785853e-12 1.3500000307092082], [1.0, 0.3599999909960635], [1.8971902887539867 0.0 0.0 0.0; 1.0541023627687969 0.2981077601504802 0.0 0.0; 0.37944752683157257 -0.5366572318946514 0.00014967827641466184 0.0; 0.9487053521430425 0.670789211874255 -0.000119788756147063 0.0])

In [43]:
## Generate worst-case function

wf = generate_worst_case_function_ratio(G_star, Ft_star, L_cholesky_star, γ_kM1_glb_opt_plshd, β_kM1_glb_opt_plshd)

[ Info: p[:,0] = Any[0.9487053521430425, 0.670789211874255, -0.000119788756147063, 0.0]
[ Info: p[:,1] = Any[0.5312297692262988, -0.4293384626661688, 0.0001305134156066089, 0.0]


worst_case_function([0.0 1.8971902887539867 0.8852052592961397; 0.0 0.0 -0.7155315808065795; 0.0 0.0 0.00012777879627680924; 0.0 0.0 0.0], [0.0 1.0541023627687969 0.37944752683157257; 0.0 0.2981077601504802 -0.5366572318946514; 0.0 0.0 0.00014967827641466184; 0.0 0.0 0.0], [0.0, 1.0, 0.3599999909960635])

In [44]:
x_array = wf.x_array 
# first column of x_array = x_⋆, 
# second column of x_array = x_0
# third column of x_array = x_1 and so on

4×3 OffsetArray(::Matrix{Float64}, 1:4, -1:1) with eltype Float64 with indices 1:4×-1:1:
 0.0  1.89719   0.885205
 0.0  0.0      -0.715532
 0.0  0.0       0.000127779
 0.0  0.0       0.0

In [45]:
g_array = wf.g_array
# first column of g_array = g_⋆, 
# second column of g_array = g_0
# third column of g_array = g_1 and so on

4×3 OffsetArray(::Matrix{Float64}, 1:4, -1:1) with eltype Float64 with indices 1:4×-1:1:
 0.0  1.0541     0.379448
 0.0  0.298108  -0.536657
 0.0  0.0        0.000149678
 0.0  0.0        0.0

In [46]:
f_array = wf.f_array
# first column of f_array = f_⋆, 
# second column of f_array = f_0
# third column of f_array = f_1 and so on

3-element OffsetArray(::Vector{Float64}, -1:1) with eltype Float64 with indices -1:1:
 0.0
 1.0
 0.3599999909960635

In [47]:
## Dual SDP by fixing the stepsize

p_star_dual, ℓ_1_norm_λ, tr_Z, λ_opt, ν_opt, τ_opt, ρ_opt, Z_opt, L_cholesky_opt_Z, idx_set_λ_effective, θ_opt = Dual_SDP_with_fixed_stepsize_ratio(
	# Inputs
	# ======
	μ, L, R, γ_kM1_glb_opt_plshd, β_kM1_glb_opt_plshd;
	# sparsifier coefficients
	# -----------------------
	c_λ = 1, c_τ = 0, c_Z = 0, c_ρ = 1,
	# Options
	# =======
	objective_type = :default, # options are :default, :find_sparse_sol
	η_2_input = η_2, # controls PRP or FR, if η = 1 => PRP, and if η = 0 => FR
	c_0_input_initial = c_0_input_initial_given, # decides ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
	show_output = :on, # options are :on and :off
	ϵ_tolerance = 1e-4,
	impose_pattern = :on, # other option is :on
	obj_val_upper_bound = p_star
	);

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 18              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 17              
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 1
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve termi

[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🐒 ] Minimizing the usual performance measure
[ Info: [🌞 ] DUALSDP-POLISHED ratio = 0.35999931450428635
[ Info: NCG-PEP Theory PRP ( f_{k+1} - f_⋆ ) / ( f_k - f_⋆ ) = 0.36


In [48]:
# We use the printed output below in the PEPit verification code (file name: "3. PEPIt_verification_Python.ipynb")
println( "mu = ", μ)
println( "n = ", 1)
println( "beta = [", β_kM1_glb_opt_plshd, "]")
println( "gamma = [", γ_kM1_glb_opt_plshd, "]")
println( "rho_LB = ", p_star)
println( "rho_UB = ", t_glb_opt_plshd)
# PLEASE NOTE THAT: eta = 1 means PRP, eta = 0 means FR, please change it depending on which data set you are loading
println("########################################################################################################################")
println("# PLEASE NOTE THAT: eta = 1 means PRP, eta = 0 means FR, please change it depending on which data set you are loading")
println("########################################################################################################################")
println( "eta = ", 1) 

mu = 0.5
n = 1
beta = [0.1599888121763658]
gamma = [1.0667010860346273]
rho_LB = 0.3599999909960635
rho_UB = 0.3600000071640856
########################################################################################################################
# PLEASE NOTE THAT: eta = 1 means PRP, eta = 0 means FR, please change it depending on which data set you are loading
########################################################################################################################
eta = 1
